# Typed Objects

In addition to the interfaces offered by `ObjectAPIClient` and `DownloadedObject`, which provides access to Geoscience Objects in a way that is agnostic to the specific object type, the `evo-objects` package also provides a set of "typed objects" that represent specific object types.

As usual, we need to first authenticate, which we can do using the `ServiceManagerWidget` from the `evo-notebooks` package

In [1]:
from evo.notebooks import ServiceManagerWidget

manager = await ServiceManagerWidget.with_auth_code(
    client_id="core-compute-tasks-notebooks",
    base_uri="https://qa-ims.bentley.com",
    discovery_url="https://int-discover.test.api.seequent.com",
).login()

ServiceManagerWidget(children=(VBox(children=(HBox(children=(Image(value=b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR…

## Creating a regular 3D grid object

To create a new 'regular-3d-grid' object, we use the `Regular3DGridData` class to define all of the properties of the grid, then pass that to the `Regular3DGrid.create` class method.

The `cell_data` and `vertex_data` properties are dataframes that define the attributes of the cells and vertices respectively. Each column in the dataframe represents a different attribute.

Note the order of the data is important, and must match the order of the cells and vertices in the grid. The cells are ordered such that the x-axis varies the fastest, then y, then z. So for a grid of size (10, 10, 5), the first 10 rows of the dataframe correspond to the cells at (0, 0, 0) to (9, 0, 0), the next 10 rows correspond to (0, 1, 0) to (9, 1, 0), and so on.

In [4]:
import numpy as np
import pandas as pd

from evo.objects.typed import Point3, Regular3DGrid, Regular3DGridData, Rotation, Size3d, Size3i

grid = Regular3DGridData(
    name="Test Grid21",
    origin=Point3(0, 0, 0),
    size=Size3i(10, 10, 5),
    cell_size=Size3d(2.5, 5, 5),
    cell_data=pd.DataFrame(
        {
            "value": np.random.rand(10 * 10 * 5),
            "cat": pd.Categorical.from_codes(np.random.choice(range(4), size=10 * 10 * 5), ["a", "b", "c", "d"]),
        }
    ),
    vertex_data=pd.DataFrame(
        {
            "elevation": np.random.rand(11 * 11 * 6),
        }
    ),
    rotation=Rotation(90, 0, 0),
)
created_grid = await Regular3DGrid.create(manager, grid)
print(f"Created regular-3d-grid object: {created_grid.metadata.url}")

Created regular-3d-grid object: https://350mt.api.integration.seequent.com/geoscience-object/orgs/829e6621-0ab6-4d7d-96bb-2bb5b407a5fe/workspaces/783b6eef-01b9-42a7-aaf4-35e153e6fcbe/objects/ad86e620-0fc5-446f-b553-86463c762e86?version=1767906778051410205


## Downloading a regular 3D grid object

To download an existing 'regular-3d-grid' object, an inspect it, you can use the `Regular3DGrid.from_reference` class method. This method takes the context and the URL of the object to download.

For the example, we will download the object we just created.

In [6]:
grid = await Regular3DGrid.from_reference(manager, created_grid.metadata.url)

print(f"Downloaded regular-3d-grid object with name: {grid.name}")
print(f"Origin: {grid.origin}")
print(f"Size: {grid.size}")
print(f"Cell Size: {grid.cell_size}")
print(f"Rotation: {grid.rotation}")
print(f"Bounding Box: {grid.bounding_box}")

Downloaded regular-3d-grid object with name: Test Grid21
Origin: Point3(x=0.0, y=0.0, z=0.0)
Size: Size3i(nx=10, ny=10, nz=5)
Cell Size: Size3d(dx=2.5, dy=5.0, dz=5.0)
Rotation: Rotation(dip_azimuth=90.0, dip=0.0, pitch=0.0)
Bounding Box: BoundingBox(min_x=0.0, min_y=-25.0, max_x=50.0, max_y=3.061616997868383e-15, min_z=0.0, max_z=25.0)


To inspect the data, `grid.cells.as_dataframe()`, and `grid.vertices.as_dataframe()` can be used to get the cell and vertex data as pandas DataFrames.

In [7]:
await grid.cells.as_dataframe()

,value,cat
0,0.848008,c
1,0.489342,d
2,0.880559,d
3,0.067766,d
4,0.188070,c
...,...,...
495,0.098823,b
496,0.283614,b
497,0.318200,d
498,0.413474,d


In [8]:
await grid.vertices.as_dataframe()

,elevation
0,0.289931
1,0.179575
2,0.293128
3,0.248778
4,0.074564
...,...
721,0.484114
722,0.730475
723,0.030539
724,0.140452


# Updating a regular 3D grid object

To update an existing 'regular-3d-grid' object, we can modify the properties of the `Regular3DGrid` instance, and then call the `update` method to submit the changes to the Geoscience Object service. This will create a new version of the object with the updated properties.

Updating the cell or vertex data can be done by calling the `set_dataframe` method on the `cells` or `vertices` attributes respectively, this will upload the new data to the Geoscience Object service, but the object will remain the same version until the `update` method is called.

In [ ]:
print(f"Version before update: {grid.metadata.version_id}")

grid.name = "Updated Test Grid"

await grid.cells.set_dataframe(
    pd.DataFrame(
        {
            "value": np.ones(10 * 10 * 5),
        }
    )
)

# Update the object to create a new version with the modified properties
await grid.update()
print(f"Updated version: {grid.metadata.version_id}")

## Working with Block Models

Block models are stored in the Block Model Service, which is separate from the Geoscience Object Service. However, you can create a reference to a block model in the Geoscience Object Service using `BlockModelRef`. This allows block models to be used in workflows that expect Geoscience Objects, such as compute tasks.

The `BlockModelRef` acts as a proxy - the geometry and attribute definitions are stored in the Geoscience Object Service, while the actual block data is stored in and accessed through the Block Model Service.

### Creating a Block Model in the Block Model Service

First, let's create a block model directly in the Block Model Service using `RegularBlockModel`:

> **Note:** If the imports below fail, you may need to add the local source packages to your Python path. Uncomment and run the cell below first.

In [3]:
# Setup for local development source - run this cell FIRST before any other imports
import sys

# Remove any cached evo modules to force reimport from local source
mods_to_remove = [key for key in sys.modules.keys() if key.startswith('evo.blockmodels')]
for mod in mods_to_remove:
    del sys.modules[mod]

# Add local source paths at the beginning
sys.path.insert(0, r"C:\Source\evo-python-sdk\packages\evo-blockmodels\src")
sys.path.insert(0, r"C:\Source\evo-python-sdk\packages\evo-objects\src")
sys.path.insert(0, r"C:\Source\evo-python-sdk\packages\evo-sdk-common\src")

print("Local source paths configured")

Local source paths configured


In [6]:
import numpy as np
import pandas as pd
from evo.blockmodels import RegularBlockModel, RegularBlockModelData, Point3 as BMPoint3, Size3i as BMSize3i, Size3d as BMSize3d, Units

# Create sample block model data
# The DataFrame must include geometry columns (i, j, k) for block indices
n_blocks = (10, 10, 5)
total_blocks = n_blocks[0] * n_blocks[1] * n_blocks[2]

# Generate block indices
indices = []
for k in range(n_blocks[2]):
    for j in range(n_blocks[1]):
        for i in range(n_blocks[0]):
            indices.append((i, j, k))

block_data = pd.DataFrame({
    "i": [idx[0] for idx in indices],
    "j": [idx[1] for idx in indices],
    "k": [idx[2] for idx in indices],
    "grade": np.random.rand(total_blocks) * 10,  # Random grade values 0-10
    "density": np.random.rand(total_blocks) * 2 + 2,  # Random density 2-4
})

# Create the block model data
# Use the Units class for valid unit IDs
bm_data = RegularBlockModelData(
    name="Example Block Model 7",
    description="A sample block model created from the notebook",
    origin=BMPoint3(0, 0, 0),
    n_blocks=BMSize3i(10, 10, 5),
    block_size=BMSize3d(2.5, 5.0, 5.0),
    cell_data=block_data,
    crs="EPSG:28354",
    size_unit_id=Units.METRES,
    units={"grade": Units.GRAMS_PER_TONNE, "density": Units.TONNES_PER_CUBIC_METRE},
)

# Create the block model in the Block Model Service
block_model = await RegularBlockModel.create(manager, bm_data)
print(f"Created block model: {block_model.id}")
print(f"Name: {block_model.name}")
print(f"Origin: {block_model.origin}")
print(f"Number of blocks: {block_model.n_blocks}")
print(f"Block size: {block_model.block_size}")
print(f"Geoscience Object ID: {block_model.metadata.geoscience_object_id}")

Created block model: d219e96a-a459-4753-a489-578d78b39fef
Name: Example Block Model 7
Origin: Point3(x=0.0, y=0.0, z=0.0)
Number of blocks: Size3i(nx=10, ny=10, nz=5)
Block size: Size3d(dx=2.5, dy=5.0, dz=5.0)
Geoscience Object ID: a86639ea-94b9-4fff-b87e-c94f4d237ed8


### Loading the Block Model Reference from the Geoscience Object Service

When a block model is created via the Block Model Service, it automatically creates a corresponding Geoscience Object reference. We can load this using `BlockModelRef.from_reference` with the `geoscience_object_id`:

In [7]:
from evo.objects.typed import BlockModelRef
from evo.objects import ObjectReference

# Load the reference that was automatically created with the block model
# The geoscience_object_id links the Block Model Service object to its Geoscience Object reference
# We need to construct the full object reference URL from the UUID
object_ref = ObjectReference.new(
    environment=manager.get_environment(),
    object_id=block_model.metadata.geoscience_object_id,
)

bm_ref = await BlockModelRef.from_reference(manager, object_ref)

print(f"Loaded BlockModelRef: {bm_ref.metadata.url}")
print(f"Block Model UUID: {bm_ref.block_model_uuid}")
print(f"Geometry type: {bm_ref.geometry.model_type}")
print(f"Origin: {bm_ref.geometry.origin}")
print(f"N blocks: {bm_ref.geometry.n_blocks}")
print(f"Block size: {bm_ref.geometry.block_size}")
print(f"Attributes: {[attr.name for attr in bm_ref.attributes]}")

Loaded BlockModelRef: https://350mt.api.integration.seequent.com/geoscience-object/orgs/829e6621-0ab6-4d7d-96bb-2bb5b407a5fe/workspaces/783b6eef-01b9-42a7-aaf4-35e153e6fcbe/objects/a86639ea-94b9-4fff-b87e-c94f4d237ed8?version=1767915195498119829
Block Model UUID: d219e96a-a459-4753-a489-578d78b39fef
Geometry type: regular
Origin: Point3(x=0.0, y=0.0, z=0.0)
N blocks: Size3i(nx=10, ny=10, nz=5)
Block size: Size3d(dx=2.5, dy=5.0, dz=5.0)
Attributes: ['grade', 'density']


### Loading an Existing Block Model Reference

You can load an existing `BlockModelRef` using `from_reference`, just like other typed objects:

In [8]:
# Load the reference we just created
loaded_ref = await BlockModelRef.from_reference(manager, bm_ref.metadata.url)

print(f"Loaded BlockModelRef: {loaded_ref.name}")
print(f"Block Model UUID: {loaded_ref.block_model_uuid}")
print(f"Bounding Box: {loaded_ref.bounding_box}")

Loaded BlockModelRef: Example Block Model 7
Block Model UUID: d219e96a-a459-4753-a489-578d78b39fef
Bounding Box: BoundingBox(min_x=0.0, min_y=0.0, max_x=25.0, max_y=50.0, min_z=0.0, max_z=25.0)


### Accessing Block Model Data

The `BlockModelRef` provides methods to access the actual block data from the Block Model Service:

In [9]:
# Get all block data as a DataFrame
df = await loaded_ref.get_data(columns=["*"])
print(f"Retrieved {len(df)} blocks")
df.head(10)

Retrieved 500 blocks


,x,y,z,density,grade
0,8.75,2.5,12.5,3.087954,6.298103
1,8.75,2.5,17.5,3.817644,0.292872
2,8.75,2.5,22.5,3.547730,2.949628
3,8.75,7.5,12.5,3.921254,7.570661
4,8.75,7.5,17.5,3.260609,0.022991
5,8.75,7.5,22.5,2.162759,1.379231
6,8.75,12.5,12.5,2.342338,4.179035
7,8.75,12.5,17.5,3.658534,9.152260
8,8.75,12.5,22.5,3.604668,4.842959
9,11.25,2.5,12.5,2.625589,5.875343


In [7]:
# You can also query specific columns
grade_data = await loaded_ref.get_data(columns=["grade"])
print(f"Grade statistics:")
print(grade_data["grade"].describe())

Grade statistics:
count    500.000000
mean       4.879904
std        2.943977
min        0.020242
25%        2.435911
50%        4.694658
75%        7.477744
max        9.985183
Name: grade, dtype: float64


### Adding New Attributes to a Block Model

You can add new attributes to the block model through the reference. This creates a new version in the Block Model Service:

In [10]:
# Calculate a new attribute based on existing data
df = await loaded_ref.get_data(columns=["grade", "density"])

# Add geometry columns required for upload
df_with_new_attr = pd.DataFrame({
    "i": [idx[0] for idx in indices],
    "j": [idx[1] for idx in indices],
    "k": [idx[2] for idx in indices],
    "metal_content": df["grade"] * df["density"],  # grade * density
})

# Add the new attribute to the block model
# Use Units class for valid unit IDs
new_version = await loaded_ref.add_attribute(
    df_with_new_attr,
    attribute_name="metal_content",
    unit=Units.KG_PER_CUBIC_METRE,
)

print(f"Added attribute 'metal_content', new version: {new_version.version_id}")

Added attribute 'metal_content', new version: 3


### Updating Multiple Attributes

For more complex updates (adding multiple columns, updating existing columns, or deleting columns), use `update_attributes`:

In [11]:
# Create data with multiple new attributes
df_updates = pd.DataFrame({
    "i": [idx[0] for idx in indices],
    "j": [idx[1] for idx in indices],
    "k": [idx[2] for idx in indices],
    "classification": np.where(df["grade"] > 5, "high", "low"),
    "value_index": df["grade"] * df["density"] * 100,
})

# Add multiple new columns at once
version = await loaded_ref.update_attributes(
    df_updates,
    new_columns=["classification", "value_index"],
    units={"value_index": "$/t"},
)

print(f"Updated block model, new version: {version.version_id}")

# Verify the new attributes
updated_df = await loaded_ref.get_data(columns=["*"])
print(f"Columns now available: {list(updated_df.columns)}")


delete=[] new=[ColumnLite(data_type=<DataType.Utf8: 'Utf8'>, title='classification', unit_id=None), ColumnLite(data_type=<DataType.Float64: 'Float64'>, title='value_index', unit_id='$/t')] rename=[] update=[] update_metadata=None
Updated block model, new version: 4
Columns now available: ['x', 'y', 'z', 'density', 'grade']
